In [16]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import pandas as pd


from PIL import Image

In [7]:
def generate_population(pop_size, nlayers, max_nfilters, max_sfilters):
    # Given the parameters returns randomly generated population

    np.random.seed(0)
    pop_nlayers = np.random.randint(1, max_nfilters, (pop_size, nlayers))
    pop_sfilters = np.random.randint(1, max_sfilters, (pop_size, nlayers))
    pop_total = np.concatenate((pop_nlayers, pop_sfilters), axis=1)
    return pop_total


def fitness(pop, X, Y, epochs):
    pop_acc = []
    for i in range(pop.shape[0]):
        nfilters = pop[i][0:3]
        sfilters = pop[i][3:]
        model = CNN(nfilters, sfilters)
        H = model.fit(X, Y, batch_size=32, epochs=epochs)
        acc = H.history['accuracy']
        pop_acc.append(max(acc))
    return pop_acc


def select_parents(pop, nparents, fitness):
    parents = np.zeros((nparents, pop.shape[1]))
    for i in range(nparents):
        best = np.argmax(fitness)
        parents[i] = pop[best]
        fitness[best] = -99999
    return parents


def crossover(parents, pop_size):
    nchild = pop_size - parents.shape[0]
    nparents = parents.shape[0]
    child = np.zeros((nchild, parents.shape[1]))
    for i in range(nchild):
        first = i % nparents
        second = (i+1) % nparents
        child[i, :2] = parents[first][:2]
        child[i, 2] = parents[second][2]
        child[i, 3:5] = parents[first][3:5]
        child[i, 5] = parents[second][5]
    return child


def mutation(child):
    for i in range(child.shape[0]):
        val = np.random.randint(1, 6)
        ind = np.random.randint(1, 4) - 1
        if child[i][ind] + val > 100:
            child[i][ind] -= val
        else:
            child[i][ind] += val
        val = np.random.randint(1, 4)
        ind = np.random.randint(4, 7) - 1
        if child[i][ind] + val > 20:
            child[i][ind] -= val
        else:
            child[i][ind] += val
    return child

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


class TwoLayeredCNN(nn.Module):
    def __init__(self, in_channels, num_classes, filter_sizes):
        super(TwoLayeredCNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels, filter_sizes[0][0], kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            filter_sizes[0][0], filter_sizes[0][1], kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(filter_sizes[0][1] * 7 * 7, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


class GeneticAlgorithm:
    def __init__(self, population_size, num_generations, mutation_rate, crossover_rate, fitness_func):
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.fitness_func = fitness_func

    def generate_population(self, num_layers, num_filters_range, filter_size_range):
        population = []
        for _ in range(self.population_size):
            chromosome = []
            for _ in range(num_layers):
                num_filters = np.random.choice(num_filters_range)
                filter_size = np.random.choice(filter_size_range)
                chromosome.append((num_filters, filter_size))
            population.append(chromosome)
        return population

    def fitness(self, chromosome):
        return self.fitness_func(chromosome)

    def select_parents(self, population):
        fitness_values = [self.fitness(chromosome)
                          for chromosome in population]
        total_fitness = sum(fitness_values)
        probabilities = [fitness / total_fitness for fitness in fitness_values]

        # Convert population to 1-dimensional array
        flat_population = [
            chromosome for sublist in population for chromosome in sublist]

        # Ensure probabilities has the same size as flat_population
        probabilities = [probabilities[i // 2]
                         for i in range(len(flat_population))]

        # Normalize probabilities to ensure they sum up to 1
        sum_probabilities = sum(probabilities)
        probabilities = [prob / sum_probabilities for prob in probabilities]

        # Select parents from the flat_population
        parents_indices = np.random.choice(
            len(flat_population), size=2, replace=False, p=probabilities)
        parents = [flat_population[index] for index in parents_indices]

        # Reshape parents to match the original population format
        parent1 = parents[:len(parents)//2]
        parent2 = parents[len(parents)//2:]
        return parent1, parent2

    def crossover(self, parent1, parent2):
        child1 = []
        child2 = []
        for i in range(len(parent1)):
            if np.random.rand() < self.crossover_rate:
                child1.append(parent2[i])
                child2.append(parent1[i])
            else:
                child1.append(parent1[i])
                child2.append(parent2[i])
        return child1, child2

    def mutation(self, chromosome):
        for i in range(len(chromosome)):
            if np.random.rand() < self.mutation_rate:
                num_filters = np.random.choice(num_filters_range)
                filter_size = np.random.choice(filter_size_range)
                chromosome[i] = (num_filters, filter_size)
        return chromosome

    def evolve(self, num_layers, num_filters_range, filter_size_range):
        population = self.generate_population(
            num_layers, num_filters_range, filter_size_range)
        for generation in range(self.num_generations):
            new_population = []
            for _ in range(self.population_size // 2):
                parent1, parent2 = self.select_parents(population)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutation(child1)
                child2 = self.mutation(child2)
                new_population.append(child1)
                new_population.append(child2)
            population = new_population
        return population[0]  # Returning the best chromosome after evolution
        
# Fitness function example (to be replaced with your own fitness function)
def fitness_func(chromosome):
    return sum([num_filters for num_filters, _ in chromosome])


# GA parameters
population_size = 20
num_generations = 100
mutation_rate = 0.1
crossover_rate = 0.8

# CNN parameters
in_channels = 3
num_classes = 10
num_layers = 2
num_filters_range = [16, 32, 64]  # Possible number of filters
filter_size_range = [3, 5, 7]  # Possible filter sizes

# Create GeneticAlgorithm instance
ga = GeneticAlgorithm(population_size, num_generations,
                      mutation_rate, crossover_rate, fitness_func)

# Evolve the population to find the best chromosome
best_chromosome = ga.evolve(num_layers, num_filters_range, filter_size_range)

# Create the CNN model with the best chromosome
model = TwoLayeredCNN(in_channels, num_classes, best_chromosome)

# Print the best chromosome and its fitness value
print("Best Chromosome:", best_chromosome)
print("Fitness Value:", ga.fitness(best_chromosome))

Best Chromosome: [(16, 5)]
Fitness Value: 16
